In [252]:
from bs4 import BeautifulSoup, Comment
import requests
import regex as re
import xml
from selenium import webdriver
import pandas as pd
import random

In [253]:
website = ["https://www.kasikornbank.com/en/rate/pages/foreign-exchange.aspx",
"https://www.scb.co.th/th/personal-banking/foreign-exchange-rates.html",
"https://www.bangkokbank.com/th-TH/Personal/Other-Services/View-Rates/Foreign-Exchange-Rates",
"https://exchangerate.krungthai.com/#/counterRate",
"https://www.krungsri.com/th/exchangerate/Todayrates"]
#ธนาคารซื้อ #ธนาคารขาย

#Remove HTML T A G S by regex |||||||||| Actually, use get_text() Function. It's much easier. =,,=

In [254]:
def remove_tag(html_str):
    TAG_RE = re.compile(r'<[^>]+>')
    text = TAG_RE.sub('!@', html_str)
    mylist = re.split(r'!@', text)
    clear_lst = []
    for clear_text in mylist:
        if (clear_text not in "\n") and (clear_text not in " ") and (clear_text not in ""):
            clear_lst.append(clear_text)
    return clear_lst
#https://www.guru99.com/python-regular-expressions-complete-tutorial.html

#start browser

In [255]:
def start_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    # executable_path param is not needed if you updated PATH
    browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')
    return browser

def end_browser(browser):
    browser.quit()


#bank

In [256]:
def kasikorn(url): #33
    browser = start_browser()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    end_browser(browser)

    #clean soup data
    data = soup.find('div',{"id" : "divTableRate"}).find('tbody').find_all('tr')
    lst_rows = []
    for index,value in enumerate(data):
        if index%2==0:
            lst = remove_tag(str(value))
            lst2 = lst[2:]
            lst_buy = [lst2[3],lst2[1],lst2[0],"-",lst2[2]]
            lst_sell = [lst2[5],lst2[4],lst2[4],lst2[4]]
            lst_rows.append([lst[0].replace(" ",""),lst_buy,lst_sell])
    
    return lst_rows

In [257]:
def scb(url): #29
    browser = start_browser()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    end_browser(browser)
    
    #clean soup data
    data = soup.find('div',{"class" : "exchange-rate"}).find('tbody').find_all('tr')
    data = data[2:]
    lst_rows = []
    for index,value in enumerate(data):
        lst = remove_tag(str(value))
        lst2 = lst[2:]
        lst_buy = [lst2[5],lst2[3],lst2[4],"-",lst2[2]]
        lst_sell = [lst2[1],lst2[0],lst2[0],"-"]
        lst_rows.append([lst[0],lst_buy,lst_sell])
    return lst_rows
    

In [258]:
def bangkok(url):
    browser = start_browser()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    end_browser(browser)

    #clean soup data
    data = soup.find('table',{"class" : "table-primary table-foreign-exchange-rates blue"}).find('tbody').find_all('tr')
    lst_rows = []
    for index,value in enumerate(data):
        clear_lst = remove_tag(str(value))
        clear_lst = [x.replace(" ", "") for x in clear_lst]
        while(len(clear_lst) < 7):
            clear_lst.append('-')
        lst2 = clear_lst[2:]
        lst_buy = [lst2[0],'-','-',lst2[2],lst2[3]]
        lst_sell = [lst2[1],"-",lst2[4],lst2[4]]
        lst_rows.append([clear_lst[0],lst_buy,lst_sell])
    return lst_rows

In [259]:
# def krungthai(url):
#     browser = start_browser()
#     browser.get(url)
#     html = browser.page_source
#     soup = BeautifulSoup(html, features="html.parser")
#     end_browser(browser)

#     #clean soup data
#     data = soup.find('table',{"id" : "cur-table-1"}).find('tbody').find_all('tr',{"class" : "ng-scope"})
#     lst_rows = []
#     for index,value in enumerate(data):
#         if index != 8 :
#             clear_lst = remove_tag(str(value))
#             clear_lst = [x.replace(" ", "") for x in clear_lst]
#             clear_lst = [x.replace("Unq", "-") for x in clear_lst]
#             lst2 = clear_lst[2:]
#             lst_buy = [lst2[3],lst2[0],'-','-',lst2[1]]
#             lst_sell = [lst2[4],lst2[2],'-','-']
#             lst_rows.append([clear_lst[0],lst_buy,lst_sell])
#     return lst_rows

In [260]:
def krungsri(url):
    browser = start_browser()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    end_browser(browser)

    #clen soup data
    data = soup.find('div',{"class" : "table-scroll"}).find('tbody').find_all('tr')
    lst_rows = []
    for index,value in enumerate(data):
        lst = remove_tag(str(value))
        remove = re.compile(r'\s|\xa0|\*')
        lst = [remove.sub('', x) for x in lst]
        lst = [x.replace("UNQ.", "-") for x in lst]
        lst2 = lst[-5:]
        lst_buy = [lst2[0],lst2[2],'-','-',lst2[3]]
        lst_sell = [lst2[1],lst2[4],lst2[4],lst2[4]]
        lst_rows.append([lst[1],lst_buy,lst_sell])
    return lst_rows

#start browser

In [261]:
kasi_data = kasikorn(website[0])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1380\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [262]:
scb_data = scb(website[1])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1380\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [263]:
bangkok_data = bangkok(website[2])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1380\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [264]:
# krungthai_data = krungthai(website[3])

In [265]:
krungsri_data = krungsri(website[4])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1380\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [266]:
kasi_data[0]

['USD1',
 ['32.3613', '33.27', '33.27', '-', '33.37'],
 ['33.71', '33.67', '33.67', '33.67']]

In [267]:
scb_data[0][1]

['33.1', '33.25', '33.15', '-', '33.35']

In [268]:
bangkok_data[0]

['USD1', ['32.23', '-', '-', '-', '-'], ['33.77', '-', '-', '-']]

In [269]:
krungsri_data[0]

['USD',
 ['32.42000', '33.18000', '-', '-', '33.28000'],
 ['33.80000', '33.78000', '33.78000', '33.78000']]

#

#test bank

In [270]:
#browser = start_browser()

In [271]:
# browser.get(website[4])
# html = browser.page_source
# soup = BeautifulSoup(html, features="html.parser")
# end_browser(browser)

#test proxy

In [272]:
# proxieszz = []

# res = requests.get('https://free-proxy-list.net/', headers={'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"})
# soup = BeautifulSoup(res.text,"lxml")
# data = soup.find("div",{"class" : "table-responsive fpl-list"}).find('tbody').find_all('tr')
# for items in data:
#     proxy_list = items.find('td').get_text()
#     port_list = items.find_all('td')[1].get_text()
#     proxieszz.append(proxy_list+":"+port_list)

In [273]:
# proxieszz

In [274]:
# while True:
#     choosenProxy = random.choice(proxieszz)
#     proxyDict = {
#         'http' : f'http://{choosenProxy}',
#         'https' : f'https://{choosenProxy}'
#     }
#     print("trying with:",proxyDict)
#     try:
#         response = requests.get('https://www.google.com/',proxies=proxieszz,timeout=5)
#         print(response.status_code)
#         break
#     except Exception:
#         continue

In [275]:
# my_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"}
# url = 'https://exchangerate.krungthai.com/'
# res = requests.get(url,headers=my_headers)
# soup = BeautifulSoup(res.text, 'html.parser')


In [276]:
#soup

#get_currency

In [277]:
def get_currency(all_bank_data):
    currency_list = []
    for d in all_bank_data:
        for index,currency in enumerate(d):
            if index >= 3 :
                currency_list.append(currency[0])
    return list(set([x for x in currency_list if currency_list.count(x) == len(all_bank_data)]))

In [278]:
all_bank_data = kasi_data,scb_data,bangkok_data,krungsri_data


In [279]:
all_bank_data[1][0][0]

'USD'

In [280]:
currency_list = get_currency(all_bank_data)

In [281]:
currency_list

['ZAR',
 'EUR',
 'CNY',
 'SAR',
 'BHD',
 'JPY',
 'AED',
 'KRW',
 'DKK',
 'NZD',
 'PHP',
 'INR',
 'VND',
 'HKD',
 'SEK',
 'TWD',
 'BND',
 'CHF',
 'MYR',
 'SGD',
 'GBP',
 'NOK',
 'IDR',
 'AUD',
 'CAD']

#pack data to csv file

In [282]:

#result = pd.concat([bank,result], axis=1, join='inner',levels=2)
#{'ราคาซื้อ':['ธนบัตร','เช็คเดินทาง','ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']}
#df[['ราคาที่ธนาคารรับซื้อ']] = pd.DataFrame()#pd.DataFrame(columns=buy_rate)

for currency in currency_list:
    ##########
    bank_name = {0 : 'ธนาคารกสิกรไทย',1 : 'ธนาคารไทยพาณิชย์',2 : 'ธนาคารกรุงเทพ',3 : 'ธนาคารกรุงศรีอยุธยา'}

    bank = pd.DataFrame()
    bank = bank.reindex(columns=['ธนาคาร'])
    #bank.columns = pd.MultiIndex.from_product(bank.columns])

    buy_rate = ['ธนบัตร', 'ตั๋วเงิน', 'เช็คเดินทาง','ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
    br = pd.DataFrame()
    br = br.reindex(columns=buy_rate)
    br.columns = pd.MultiIndex.from_product([["ราคาที่ธนาคารรับซื้อ"], br.columns])

    sell_rate = ['ธนบัตร', 'เช็คเดินทาง', 'ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
    sr = pd.DataFrame()
    sr = sr.reindex(columns=sell_rate)
    sr.columns = pd.MultiIndex.from_product([["ราคาที่ธนาคารขาย"], sr.columns])

    result = pd.concat([br,sr], axis=1, join='inner')
    result = pd.concat([bank,result], axis=1, join='inner')##########################
    

    for index,bank in enumerate(all_bank_data):
        data_list = []       
        for currency_data in bank:
            if currency_data[0] == currency:
                data_list = [bank_name[index]]+currency_data[1]+currency_data[2]
                break
        result.loc[len(result)] = data_list


    result.to_csv('bank_buy_sell_rate/'+str(currency)+'.csv',index=False)

In [283]:
result

,ธนาคาร,"(ราคาที่ธนาคารรับซื้อ, ธนบัตร)","(ราคาที่ธนาคารรับซื้อ, ตั๋วเงิน)","(ราคาที่ธนาคารรับซื้อ, เช็คเดินทาง)","(ราคาที่ธนาคารรับซื้อ, ตั๋วแลกเงิน&ดราฟ)","(ราคาที่ธนาคารรับซื้อ, โอนเงินทางโทรเลข/โอนเงิน)","(ราคาที่ธนาคารขาย, ธนบัตร)","(ราคาที่ธนาคารขาย, เช็คเดินทาง)","(ราคาที่ธนาคารขาย, ตั๋วแลกเงิน&ดราฟ)","(ราคาที่ธนาคารขาย, โอนเงินทางโทรเลข/โอนเงิน)"
0,ธนาคารกสิกรไทย,26.01914,26.36318,26.36318,-,26.43796,27.15326,27.01897,27.01897,27.01897
1,ธนาคารไทยพาณิชย์,25.88125,26.3025,26.2225,-,26.4325,27.1525,27.08875,27.08875,-
2,ธนาคารกรุงเทพ,26.07,-,-,26.34000,26.40000,27.24,-,27.13500,27.13500
3,ธนาคารกรุงศรีอยุธยา,26.07000,26.31512,-,-,26.41512,27.13000,27.11512,27.11512,27.11512


In [284]:
# buy_rate = ['ธนบัตร', 'ตั๋วเงิน', 'เช็คเดินทาง','ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
# br = pd.DataFrame()
# br = br.reindex(columns = buy_rate)

In [285]:
# sell_rate = ['ธนบัตร', 'เช็คเดินทาง', 'ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
# sr = pd.DataFrame()
# sr = sr.reindex(columns = sell_rate)

In [286]:
# br

In [287]:
# sr

In [288]:
# for i,d in enumerate(kasi_data):
#     #print(d[0]+d[1])
#     dd = d[1]
#     br.loc[len(br)] = dd

In [289]:
# br

In [290]:
#df.to_csv('thb-usd.csv',index=False)

In [291]:
# f = open('text.txt', 'w')
# with open('text.txt', 'w',encoding='utf-8') as f:
#     f.write(str(soup))

In [292]:
#soup